In [1]:
# sample structures from each committor slice for visualization

In [1]:
import sys
import os
import importlib

import numpy as np
import scipy

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
import prettypyplot as pplt
from cycler import cycler

import pyemma
import mdtraj as md

import ivac
import extq
from extq.stop import forward_stop
import kmedoids

In [2]:
sys.path.insert(1, "../../python")
sys.path.insert(1, "../../..")
import util
import plotting

In [3]:
pplt.load_cmaps()
plt.style.use("custom")  # custom style sheet
plt.style.use("muted")  # muted color theme from SciencePlots
colors = mpl.colors.to_rgba_array(
    [
        "#364B9A",
        "#4A7BB7",
        "#6EA6CD",
        "#98CAE1",
        "#C2E4EF",
        "#EAECCC",
        "#FEDA8B",
        "#FDB366",
        "#F67E4B",
        "#DD3D2D",
        "#A50026",
    ]
)
cm_div = mpl.colors.LinearSegmentedColormap.from_list("diverging", colors)
mpl.colormaps.register(cm_div, force=True)
feature_colors = ["grey", "#E46C0A", "#51BD52", "#D85BD8", "#5AA5A5", "#0000FF", "#ffcc00"]

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
DATA_DIR = "/project/dinner/scguo/ci-vsd/data"
# S4 translocation/rotation data
cv_trajs = list(
    np.load(f"{DATA_DIR}/raw_feat/cv_dist_spin_anton.npy", allow_pickle=True)
)
cv_trajs.extend(np.load(f"{DATA_DIR}/raw_feat/cv_dist_spin_anton2.npy")[:115])
cv_arr = np.concatenate(cv_trajs)
# salt bridge distances for states
sb_trajs = list(np.load(f"{DATA_DIR}/raw_feat/feat2_raw_anton.npy", allow_pickle=True))
sb_trajs.extend(np.load(f"{DATA_DIR}/raw_feat/feat2_raw_anton2.npy")[:115])
sb_arr = np.concatenate(sb_trajs)
sb_models = np.load(f"{DATA_DIR}/models_centroids_feat2.npy")
# distances to F161
rf161 = list(np.load(f"{DATA_DIR}/raw_feat/rf161.npy", allow_pickle=True))
rf161.extend(np.load(f"{DATA_DIR}/raw_feat/rf161_anton2.npy")[:115])
rf161_arr = np.concatenate(rf161)
# committors
lag = 500
qp_du = np.load(f"{DATA_DIR}/feat2_dist_du_anton2/qp_downup_3.npy", allow_pickle=True)[8] # 50 ns lag time
qm_du = np.load(f"{DATA_DIR}/feat2_dist_du_anton2/qm_downup_3.npy", allow_pickle=True)[8] # 50 ns lag time
# weights
weights = np.load(f"{DATA_DIR}/feat2_dist_du_anton2/weights_3_feat5ivac.npy", allow_pickle=True)[0] # 0.1 ns lag time

In [6]:
feature_names = []
feature_names.append('Translocation')
feature_names.append('Rotation')
for r in ("R217", "R223", "R226", "R229", "R232"):
    for n in ("D129", "D136", "D151", "D164", "E183", "D186"):
        if n.startswith("D"):
            feature_names.append(f"{r} C$_\\zeta$-{n} C$_\\gamma$")
        else:
            feature_names.append(f"{r} C$_\\zeta$-{n} C$_\\delta$")
for i in ("R223", "R226", "R229"):
    feature_names.append(f"{i}-F161")

In [7]:
# no c-alpha distances
X = np.hstack((cv_arr, sb_arr[:, 30:], rf161_arr))
y = np.concatenate(qp_du)
print(X.shape, y.shape)

(4150115, 35) (4150115,)


# Divide into bins of 0.05 and then weight by $\pi$

In [8]:
topfile = "/project/dinner/scguo/ci-vsd/civsd-pro.psf"

In [9]:
np.random.seed(123)

In [10]:
def sample_inds(q, weights, size=None, qstep=0.05, low=0, hi=1):
    q_arr = np.concatenate(q)
    w_arr = np.concatenate(weights)
    nsteps = round((hi - low) / qstep)
    all_inds = []
    steps = np.linspace(low, hi - qstep, nsteps)
    for i, s in enumerate(steps):
        q_inds = ((q_arr >= s) & (q_arr <= s + qstep)).nonzero()[0]
        q_w = w_arr[q_inds] / np.sum(w_arr[q_inds])
        slice_inds = np.random.choice(q_inds, size=size, replace=False, p=q_w)
        all_inds.append(slice_inds)
    return steps, all_inds

In [11]:
steps, q_inds = sample_inds(qp_du, weights, size=10, qstep=0.05)

In [12]:
anton_files = []
for i in range(3, 119):
    if i == 82:
        continue
    anton_files.append(f"/project/dinner/scguo/ci-vsd/anton2/prot/civsd.prot.{i}.xtc")

In [13]:
def traj_frame(close_id):
    if close_id < 3_000_000:
        ix = util.anton_frame(close_id)
        file = f"/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-{ix[0]}.xtc"
        frame = ix[1]
        print(f"{file}\t{frame}")
        return file, frame
    else:
        ix = ((close_id - 3_000_000) // 10001, (close_id - 3_000_000) % 10001)
        print(f"{anton_files[ix[0]]}\t{ix[1]}")
        return anton_files[ix[0]], ix[1]

In [14]:
def save_frames(file_names, indices, top_file, outfile):
    print(f"Topology file: {top_file}")
    print(f"Output file: {outfile}")

    xyz, angles, lengths = [], [], []
    for (f, idx) in zip(file_names, indices):
        frame = md.load_frame(f, idx, top=top_file)
        # if mdtraj encounters an impossible frame index it just
        # returns a 0-length trajectory
        if len(frame) != 0:
            xyz.append(frame.xyz)
            angles.append(frame.unitcell_angles)
            lengths.append(frame.unitcell_lengths)

    top = md.load_topology(top_file)
    traj = md.Trajectory(
        np.concatenate(xyz),
        top,
        unitcell_angles=np.concatenate(angles),
        unitcell_lengths=np.concatenate(lengths),
    )
    traj.save(outfile)

In [15]:
files, indices = [], []
for k, inds in enumerate(q_inds):
    for i in inds:
        f, ix = traj_frame(i)
        files.append(f)
        indices.append(ix)

/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-179.xtc	11947
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-103.xtc	8060
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-58.xtc	9356
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-166.xtc	2396
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-179.xtc	48011
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-154.xtc	4195
/project/dinner/scguo/ci-vsd/anton2/prot/civsd.prot.100.xtc	8325
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-179.xtc	3
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-159.xtc	3840
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-152.xtc	794
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-154.xtc	6888
/project/dinner/scguo/ci-vsd/anton2/prot/civsd.prot.38.xtc	2782
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-172.xtc	5353
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-11.xtc	6974
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-165.xtc	4724
/project/dinner/scguo/ci-vsd/anton2/prot/civsd.prot.40.xtc	6739
/beagle3/dinner/sc

In [44]:
save_frames(files, indices, topfile, f"../../data/q_bin/q_05_10_pi.dcd")

Topology file: /project/dinner/scguo/ci-vsd/civsd-pro.psf
Output file: ../../data/q_bin/q_05_10_pi.dcd


# Weight by reactive density

In [16]:
piqmqp = [w * qp.clip(min=0, max=1) * qm.clip(min=0, max=1) for (w, qp, qm) in zip(weights, qp_du, qm_du)]
steps, q_inds = sample_inds(qp_du, piqmqp, size=10, qstep=0.05)

In [18]:
files, indices = [], []
for k, inds in enumerate(q_inds):
    for i in inds:
        f, ix = traj_frame(i)
        files.append(f)
        indices.append(ix)

/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-166.xtc	3504
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-6.xtc	3398
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-175.xtc	6043
/project/dinner/scguo/ci-vsd/anton2/prot/civsd.prot.116.xtc	127
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-180.xtc	46506
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-168.xtc	8277
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-11.xtc	3635
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-179.xtc	21818
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-176.xtc	2785
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-4.xtc	3953
/project/dinner/scguo/ci-vsd/anton2/prot/civsd.prot.59.xtc	313
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-179.xtc	63432
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-172.xtc	4469
/project/dinner/scguo/ci-vsd/anton2/prot/civsd.prot.58.xtc	5205
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-169.xtc	1451
/beagle3/dinner/scguo/anton-old/xtc1000ns/civsd-179.xtc	23760
/beagle3/dinner/s

In [48]:
save_frames(files, indices, topfile, f"../../data/q_bin/q_05_10_piqpqm.dcd")

Topology file: /project/dinner/scguo/ci-vsd/civsd-pro.psf
Output file: ../../data/q_bin/q_05_10_piqpqm.dcd


# Interpolate frames

In [33]:
traj1 = md.load("../../data/q_bin/q_05_10_piqpqm.dcd", top=topfile)
s1s3_backbone = traj1.top.select("resid 1 to 214 and backbone")
# align backbone of helices S1-S3
traj1.superpose(traj1, frame=0, atom_indices=s1s3_backbone)
traj2 = md.load("../../data/q_bin/q_05_10_pi.dcd", top=topfile)
traj2.superpose(traj2, frame=0, atom_indices=s1s3_backbone)

<mdtraj.Trajectory with 200 frames, 2192 atoms, 134 residues, and unitcells at 0x7ff0121abe80>

In [26]:
def morph_linear(t, N):
    return t / N

def morph_cycle(t, N):
    return (1 - np.cos(np.pi * t / (N + 1))) / 2

def morph_sin2(t, N):
    return np.sqrt(np.sin(np.pi * t / N / 2))
    
def interpolate(xyz, N, morph_fn):
    """Divide each frame into N intermediate morphs
    using interpolated using morph_fn.
    
    xyz : (T, 3) array of coordinates
    N : number of intermediate frames
    
    Returns
    -------
    new_xyz : ((T - 1) * N + 1, 3) array of coordinates
    """
    new_xyz = []
    for i in range(len(xyz) - 1):
        for t in range(N):
            # interpolation value
            f = morph_fn(t, N)
            intermediate_frame = xyz[i] * (1 - f) + xyz[i + 1] * f
            new_xyz.append(intermediate_frame)
    new_xyz.append(xyz[-1])
    return new_xyz

In [37]:
morphed_traj1 = interpolate(traj1.xyz[0::10], 10, morph_linear)

In [38]:
top = md.load_topology(topfile)
traj = md.Trajectory(
    morphed_traj1,
    top
)
traj.save("../../data/q_bin/q_05_10_piqpqm_morph.xtc")